In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

In [2]:
data = pd.read_excel('2022_survey_results.xlsx', sheet_name="Responses")
data.head()

,contributors,in_US,currency,political_scale,ethnicity1,gender1,age_range1,relationship_status1,children,num_children,...,exp_education,exp_properties_business,exp_other,exp_total,income_wages,income_self,income_properties_business,income_other,income_total,unique
0,1,No,Australian Dollars (AUD),NaN,White,Male,31-35,"In a relationship, but not married","Do not have children, am undecided about havin...",NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN
1,1,No,Euros (EUR),NaN,White,Male,21-25,"Single, never married","Do not have children, am undecided about havin...",NaN,...,30.0,NaN,NaN,3840,NaN,6000.0,2000.0,NaN,8000,NaN
2,1,Yes,United States Dollars (USD),62.0,White,Male,31-35,Married,Have children,3.0,...,NaN,NaN,NaN,40000,93750.0,1500.0,0.0,16000.0,111250,NaN
3,2,Yes,United States Dollars (USD),55.0,White,Male,36-40,Divorced - Remarried,Have children,2.0,...,NaN,NaN,NaN,125000,425000.0,NaN,NaN,NaN,425000,NaN
4,2,Yes,United States Dollars (USD),20.0,"White,Asian",Non-binary,26-30,Married,"Do not have children, am undecided about havin...",NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN


In [3]:
data.shape

(1365, 118)

In [4]:
data = data.drop(data[data['currency'].isna()].index, axis=0)
data['currency'].unique()


array(['Australian Dollars (AUD)', 'Euros (EUR)',
       'United States Dollars (USD)', 'Canadian Dollars (CAD)',
       'Other (Please specify) - CHF',
       'Other (Please specify) - Swedish Kronor (SEK)',
       'British Pound Sterling (GBP)',
       'Other (Please specify) - Malaysian Ringgit (MYR)',
       'Other (Please specify) - PEN', 'Other (Please specify)',
       'Other (Please specify) - Japanese Yen (JPY)',
       'Other (Please specify) - PLN', 'Other (Please specify) - BRL',
       'Other (Please specify) - SGD', 'Other (Please specify) - HUF',
       'Other (Please specify) - Bolívares Soberanos (VEF)',
       'Other (Please specify) - INR',
       'Other (Please specify) - Philippine Pesos'], dtype=object)

In [5]:
data = data.drop(data[data['currency'] == 'Other (Please specify)'].index, axis=0)

In [6]:
ExRate = {i:0 for i in data.currency.unique()}

ExRate

{'Australian Dollars (AUD)': 0,
 'Euros (EUR)': 0,
 'United States Dollars (USD)': 0,
 'Canadian Dollars (CAD)': 0,
 'Other (Please specify) - CHF': 0,
 'Other (Please specify) - Swedish Kronor (SEK)': 0,
 'British Pound Sterling (GBP)': 0,
 'Other (Please specify) - Malaysian Ringgit (MYR)': 0,
 'Other (Please specify) - PEN': 0,
 'Other (Please specify) - Japanese Yen (JPY)': 0,
 'Other (Please specify) - PLN': 0,
 'Other (Please specify) - BRL': 0,
 'Other (Please specify) - SGD': 0,
 'Other (Please specify) - HUF': 0,
 'Other (Please specify) - Bolívares Soberanos (VEF)': 0,
 'Other (Please specify) - INR': 0,
 'Other (Please specify) - Philippine Pesos': 0}

In [7]:
ExRateValues = [0.67, 1.07, 1, 0.74, 1.1, 0.092, 1.24, 0.22, 0.27, 0.0072, 0.24, 0.2, 0.74, 0.0029, 0.000000377422, 0.012, 0.018]

ExRate = {k:v for k,v in zip(ExRate.keys(), ExRateValues)}

ExRate

{'Australian Dollars (AUD)': 0.67,
 'Euros (EUR)': 1.07,
 'United States Dollars (USD)': 1,
 'Canadian Dollars (CAD)': 0.74,
 'Other (Please specify) - CHF': 1.1,
 'Other (Please specify) - Swedish Kronor (SEK)': 0.092,
 'British Pound Sterling (GBP)': 1.24,
 'Other (Please specify) - Malaysian Ringgit (MYR)': 0.22,
 'Other (Please specify) - PEN': 0.27,
 'Other (Please specify) - Japanese Yen (JPY)': 0.0072,
 'Other (Please specify) - PLN': 0.24,
 'Other (Please specify) - BRL': 0.2,
 'Other (Please specify) - SGD': 0.74,
 'Other (Please specify) - HUF': 0.0029,
 'Other (Please specify) - Bolívares Soberanos (VEF)': 3.77422e-07,
 'Other (Please specify) - INR': 0.012,
 'Other (Please specify) - Philippine Pesos': 0.018}

In [8]:
range(78,116)

range(78, 116)

In [9]:
monetaryColumns = [52, 55, 61, 62, 69, 71, 72] + [i for i in range(78,117)]

monetaryColumns

[52,
 55,
 61,
 62,
 69,
 71,
 72,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116]

In [10]:
dataUSD = data.copy()

In [11]:
for k,v in ExRate.items():
    d = dataUSD[dataUSD.currency == k]
    d.iloc[:,monetaryColumns] = d.iloc[:,monetaryColumns] * v
    dataUSD[dataUSD.currency == k] = d

C:\Users\H487136\AppData\Local\Temp\ipykernel_4484\1161783596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.iloc[:,monetaryColumns] = d.iloc[:,monetaryColumns] * v
C:\Users\H487136\AppData\Local\Temp\ipykernel_4484\1161783596.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  d.iloc[:,monetaryColumns] = d.iloc[:,monetaryColumns] * v


In [12]:
numerical = dataUSD.select_dtypes(exclude=object)
numerical = numerical.drop(['employ_status_parttime_reg3', 'employ_status_parttime_irreg3'], axis=1)
numerical.columns

Index(['political_scale', 'num_children', 'num_future_children', 'FI_number',
       'pct_FI_number', 'RE_number', 'SWR', 'expected_annual_income',
       'expected_annual_expenses', 'isRE_FI_number', 'isRE_RE_number',
       'isRE_withdrawal_rate', 'isRE_withdrawal_amount',
       'AV_primary_residence', 'AV_brokerage', 'AV_retirement', 'AV_cash',
       'AV_dedicated_savings', 'AV_speculative', 'AV_properties_business',
       'AV_other', 'AV_total', 'debt_student', 'debt_mortgage', 'debt_auto',
       'debt_creditcard', 'debt_medical', 'debt_properties_business',
       'debt_other', 'debt_total', 'exp_not_categorized', 'exp_housing',
       'exp_utilities', 'exp_transportation', 'exp_necessities',
       'exp_luxuries', 'exp_children', 'exp_debt_repay', 'exp_tax_adv_savings',
       'exp_non_tax_adv_savings', 'exp_charity', 'exp_healthcare', 'exp_taxes',
       'exp_education', 'exp_properties_business', 'exp_other', 'exp_total',
       'income_wages', 'income_self', 'income_proper

In [13]:
categorical = dataUSD.select_dtypes(object)
categorical = categorical.drop(['isRE_misconceptions','unique'], axis=1)
categorical.columns

Index(['contributors', 'in_US', 'currency', 'ethnicity1', 'gender1',
       'age_range1', 'relationship_status1', 'children', 'ethnicity2',
       'gender2', 'age_range2', 'relationship_status2', 'ethnicity3',
       'gender3', 'age_range3', 'relationship_status3', 'industry1',
       'corporation1', 'role1', 'job_title1', 'employ_status_fulltime1',
       'employ_status_parttime_reg1', 'employ_status_parttime_irreg1',
       'employ_status_notemployed1', 'industry2', 'corporation2', 'role2',
       'job_title2', 'employ_status_fulltime2', 'employ_status_parttime_reg2',
       'employ_status_parttime_irreg2', 'employ_status_notemployed2',
       'industry3', 'corporation3', 'role3', 'job_title3',
       'employ_status_fulltime3', 'employ_status_notemployed3', 'edu_status1',
       'highest_education1', 'edu_status2', 'highest_education2',
       'edu_status3', 'highest_education3', 'housing_status', 'CoL_index',
       'is_FI', 'is_RE', 'sup_income_gov', 'sup_income_private_pension',
 

In [14]:
pd.set_option("display.max_columns", 100)

numerical.describe()

,political_scale,num_children,num_future_children,FI_number,pct_FI_number,RE_number,SWR,expected_annual_income,expected_annual_expenses,isRE_FI_number,isRE_RE_number,isRE_withdrawal_rate,isRE_withdrawal_amount,AV_primary_residence,AV_brokerage,AV_retirement,AV_cash,AV_dedicated_savings,AV_speculative,AV_properties_business,AV_other,AV_total,debt_student,debt_mortgage,debt_auto,debt_creditcard,debt_medical,debt_properties_business,debt_other,debt_total,exp_not_categorized,exp_housing,exp_utilities,exp_transportation,exp_necessities,exp_luxuries,exp_children,exp_debt_repay,exp_tax_adv_savings,exp_non_tax_adv_savings,exp_charity,exp_healthcare,exp_taxes,exp_education,exp_properties_business,exp_other,exp_total,income_wages,income_self,income_properties_business,income_other,income_total
count,1072.000000,337.000000,643.000000,8.300000e+02,830.000000,9.010000e+02,920.000000,6.650000e+02,886.000000,1.070000e+02,1.600000e+01,15.000000,15.000000,8.050000e+02,8.610000e+02,8.840000e+02,8.690000e+02,638.000000,5.990000e+02,5.210000e+02,4.140000e+02,1.298000e+03,534.000000,6.840000e+02,516.000000,494.000000,440.000000,4.740000e+02,418.000000,1.298000e+03,336.000000,465.000000,448.000000,451.000000,456.000000,449.000000,281.000000,253.000000,422.000000,3.910000e+02,302.000000,365.000000,4.110000e+02,229.000000,221.000000,193.000000,1.298000e+03,7.600000e+02,2.440000e+02,2.340000e+02,248.000000,1.298000e+03
mean,36.467351,1.738872,1.300156,2.277848e+06,28.662106,3.045430e+06,3.695848,7.270180e+04,87636.148307,2.089616e+06,2.544627e+06,1.983333,53733.333333,4.000867e+05,3.548221e+05,3.895907e+05,6.608979e+04,21427.701724,1.784236e+04,1.673454e+05,3.320089e+04,8.895935e+05,11989.284644,2.362330e+05,7206.273256,1769.157895,42.045455,3.914764e+04,1325.870813,1.476940e+05,118767.944226,25909.944796,3502.634196,4495.835255,9676.333114,12770.009621,3490.566192,2343.002925,38773.580142,4.876415e+04,2797.057550,3466.982849,5.497532e+04,947.952052,1947.022624,2512.772850,9.902721e+04,2.132223e+05,5.068539e+04,2.009257e+04,11058.330081,1.401081e+05
std,21.820866,0.908001,1.072940,1.921933e+06,21.030467,3.441423e+06,0.914274,2.259353e+05,66231.941751,1.734689e+06,1.742499e+06,1.349580,54334.238964,4.347250e+05,1.128847e+06,4.591984e+05,1.091009e+05,52993.645484,1.655810e+05,1.198010e+06,1.355743e+05,1.643376e+06,35143.610525,2.845799e+05,18642.877512,6427.552330,369.609038,2.489387e+05,14311.815141,3.116641e+05,119288.053205,21399.499621,2581.175067,6245.151322,7859.444120,12369.308258,8753.002947,8556.763604,26165.392031,1.280202e+05,7261.665886,4174.459298,1.035479e+05,3221.640324,6549.969517,5902.151916,1.724260e+05,1.704472e+05,2.597619e+05,1.456486e+05,32026.858401,2.325406e+05
min,0.000000,1.000000,0.000000,2.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,2.900000e+01,7.500000e+05,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,20.000000,1.000000,0.000000,1.300000e+06,10.000000,1.500000e+06,3.322500,2.000000e+04,50000.000000,1.000000e+06,1.450000e+06,1.235000,19000.000000,0.000000e+00,3.000000e+04,1.103750e+05,1.554000e+04,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,40000.000000,13000.000000,1746.500000,1500.000000,4485.000000,4772.200000,0.000000,0.000000,22000.000000,6.000000e+03,0.000000,685.000000,1.700000e+04,0.000000,0.000000,0.000000,0.000000e+00,1.109262e+05,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
50%,32.000000,2.000000,1.000000,2.000000e+06,25.000000,2.200000e+06,3.500000,3.600000e+04,75000.000000,1.650000e+06,2.000000e+06,

In [15]:
categorical.describe()

,contributors,in_US,currency,ethnicity1,gender1,age_range1,relationship_status1,children,ethnicity2,gender2,age_range2,relationship_status2,ethnicity3,gender3,age_range3,relationship_status3,industry1,corporation1,role1,job_title1,employ_status_fulltime1,employ_status_parttime_reg1,employ_status_parttime_irreg1,employ_status_notemployed1,industry2,corporation2,role2,job_title2,employ_status_fulltime2,employ_status_parttime_reg2,employ_status_parttime_irreg2,employ_status_notemployed2,industry3,corporation3,role3,job_title3,employ_status_fulltime3,employ_status_notemployed3,edu_status1,highest_education1,edu_status2,highest_education2,edu_status3,highest_education3,housing_status,CoL_index,is_FI,is_RE,sup_income_gov,sup_income_private_pension,sup_income_realestate,sup_income_inheritance,FI_type,RE_when_FI,expected_RE_age,FI_impact_climate,FI_impact_inflation,FI_impact_politics,isRE_FI_type,isRE_withdrawals_lessthan,isRE_withdrawals_equal,isRE_withdrawals_greaterthan
count,1298,1298,1298,1269,1265,1272,1270,1271,604,600,603,602,6,6,6,6,1225,1225,1218,1058,1152,53,81,38,568,565,564,475,509,36,20,19,3,2,2,3,2,1,1204,1207,569,570,4,4,1114,1111,976,116,646,144,163,281,940,940,942,925,931,926,114,6,12,3
unique,4,2,17,31,3,10,7,4,17,3,11,8,6,2,5,3,23,5,13,612,3,3,3,3,23,5,13,295,3,2,2,3,3,2,2,3,2,1,3,9,3,8,1,3,5,14,2,2,3,2,5,4,7,4,11,10,10,9,6,2,2,2
top,1,Yes,United States Dollars (USD),White,Male,31-35,Married,Have children,White,Female,31-35,Married,Southeast Asian,Male,61-65,Married,Information Technology (incl. software enginee...,Private corporation,Information technology,Software Engineer,For an organization,For an organization,For myself,"For an organization,For myself",Healthcare,Private corporation,Professional (licensed / certificated),Teacher,For an organization,For an organization,For myself,For myself,Information Technology (incl. software),Public agency,Information technology,IT Professional,For an organization,For myself,Not a student,Bachelor's Degree,Not a student,Bachelor's Degree,Not a student,Less than high school,Own,81-90,No,No,At some point after retirement,At some point after retirement,Immediately on retirement,At some point after retirement,FI,Partially,46-50,No change,No change,No change,FI,The amount I originally planned for,The amount I originally planned for,The average amount I plan to spend going forward
freq,670,1162,1184,924,993,379,623,340,434,413,207,507,1,3,2,4,389,564,294,77,1103,34,58,18,125,229,170,23,480,27,13,8,1,1,1,1,1,1,1135,650,540,245,4,2,651,377,859,98,589,113,100,266,475,317,216,791,440,737,57,4,8,2


In [16]:
px.histogram(categorical.in_US.values, height=500, width = 750, labels={'value': "US Residents", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=["<20"] + sorted(categorical.age_range1.dropna().unique())[:-1])

In [17]:
px.histogram(categorical.contributors.values, height=500, width = 750, labels={'value': "Number of Contributors", 'count': ""}).update_layout(showlegend=False)

In [18]:
px.histogram(categorical.relationship_status1.values, height=500, width = 750, labels={'value': "Relationship Status (Primary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [19]:
px.histogram(categorical.age_range1.values, height=500, width = 750, labels={'value': "Age Range (Primary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=["<20"] + sorted(categorical.age_range1.dropna().unique())[:-1])

In [20]:
px.histogram(categorical[categorical.is_FI == 'Yes'].age_range1, height=500, width = 750, labels={'value': "Age Range (Primary, FI)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=["<20"] + sorted(categorical.age_range1.dropna().unique())[:-1])

In [21]:
px.histogram(categorical.gender1.values, height=500, width = 750, labels={'value': "Gender (Primary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [22]:
px.histogram(categorical.ethnicity1.values, height=750, width = 750, labels={'value': "Ethnicity (Primary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [23]:
px.histogram(categorical.age_range2.values, height=500, width = 750, labels={'value': "Age Range (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=["<20"] + sorted(categorical.age_range2.dropna().unique())[:-1])

In [24]:
px.histogram(categorical[categorical.is_FI == "Yes"].age_range2, height=500, width = 750, labels={'value': "Age Range (Secondary, FI)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=["<20"] + sorted(categorical.age_range2.dropna().unique())[:-1])

In [25]:
px.histogram(categorical.gender2.values, height=500, width = 750, labels={'value': "Gender (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [26]:
px.histogram(categorical.ethnicity2.values, height=600, width = 750, labels={'value': "Ethnicity (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [27]:
px.histogram(categorical.children.values, height=500, width = 750, labels={'value': "Have/plan to have children", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=sorted(categorical.children.dropna().unique()))

In [28]:
px.histogram(numerical.num_children.values, height=500, width = 750, labels={'value': "Number of Children", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=sorted(numerical.num_children.dropna().unique()))

In [29]:
px.histogram(numerical.num_future_children.values, height=500, width = 750, labels={'value': "Number of Future Children", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=sorted(numerical.num_future_children.dropna().unique()))

In [30]:
px.histogram(categorical.highest_education1.values, height=500, width = 750, labels={'value': "Highest level of education (Primary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [31]:
px.histogram(categorical.highest_education2.values, height=500, width = 750, labels={'value': "Highest level of education (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [32]:
px.histogram(categorical.CoL_index.values, height=500, width = 750, labels={'value': "Cost of Living Index", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", 
                                                                                                                 categoryarray=["1-20"] + sorted(categorical.CoL_index.dropna().unique())[6:] + 
                                                                                                                 sorted(categorical.CoL_index.dropna().unique())[1:6])

In [33]:
px.histogram(categorical.housing_status.values, height=500, width = 750, labels={'value': "Type of Residence", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [34]:
px.histogram(categorical.industry2.values, height=600, width = 750, labels={'value': "Employment Sector (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [35]:
px.histogram(categorical.corporation2.values, height=500, width = 750, labels={'value': "Company Type (Secondary)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="total descending")

In [36]:
px.histogram(categorical.is_FI.values, height=500, width = 750, labels={'value': "Are you financially independent?", 'count': ""}).update_layout(showlegend=False)

In [37]:
px.histogram(categorical.is_RE.values, height=500, width = 750, labels={'value': "Have you retired?", 'count': ""}).update_layout(showlegend=False)

In [38]:
px.histogram(categorical.FI_type.values, height=500, width = 750, labels={'value': "Type of FI (Non-FI)", 'count': ""}).update_layout(showlegend=False)

In [39]:
px.histogram(categorical.RE_when_FI.values, height=500, width = 750, labels={'value': "Retire once Financially Independent", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder='array',
                                                                                                                                                                                        categoryarray=['Yes', 'No', 'Partially', 'Undecided'])

In [40]:
px.histogram(categorical.expected_RE_age.values, height=500, width = 750, labels={'value': "Expected Retirement Age (Non-FI)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=sorted(categorical.expected_RE_age.dropna().unique()))

In [41]:
px.histogram(categorical[categorical.is_FI == "Yes"].expected_RE_age.values, height=500, width = 750, labels={'value': "Expected Retirement Age (FI)", 'count': ""}).update_layout(showlegend=False).update_xaxes(categoryorder="array", categoryarray=sorted(categorical[categorical.is_FI == 'Yes'].expected_RE_age.dropna().unique()))

In [52]:
flag = (dataUSD.exp_total > 0) & (dataUSD.exp_total <= dataUSD.exp_total.quantile(0.99))
px.histogram(dataUSD[flag].exp_total, height=500, width=750, labels={'value': "Total Expenses", 'count': ""}).update_layout(showlegend=False)

In [43]:
flag = (dataUSD.FI_number <= dataUSD.FI_number.quantile(0.99)) & (dataUSD.FI_number > 0)
px.histogram(dataUSD[flag].FI_number,
            height=500, width=750, labels={'value': "Financial Independence Number (Non-FI)", 'count': ""}).update_layout(showlegend=False)

In [44]:
flag = (numerical.RE_number <= numerical.RE_number.quantile(0.98)) & (numerical.RE_number > 0)
px.histogram(numerical[flag].RE_number, height=500, width=750, labels={'value': "Early Retirement Number (Non-FI)", 'count': ""}).update_layout(showlegend=False)

In [45]:
flag = (numerical.SWR <= numerical.SWR.quantile(0.99)) & (numerical.SWR > 0)
px.histogram(numerical[flag].SWR, height=500, width=750, nbins=10, labels={'value': "Safe Withdrawal Rate (Non-FI)", 'count': ""}).update_layout(showlegend=False)

In [46]:
px.histogram(numerical.pct_FI_number, height=500, width=750, labels={'value': "% FI", 'count': ""}).update_layout(showlegend=False)

In [47]:
px.histogram(numerical.political_scale, height=500, width=750, labels={'value': "Political Lean", 'count': ""}).update_layout(showlegend=False)

In [48]:
px.histogram(numerical.expected_annual_income[(numerical.expected_annual_income <= numerical.expected_annual_income.quantile(0.99)) & (numerical.expected_annual_income > 0)],
 height=500, width=750, labels={'value': "Annual Income in Retirement (Expected)", 'count': ""}).update_layout(showlegend=False)

In [49]:
flag = (numerical.expected_annual_expenses <= numerical.expected_annual_expenses.quantile(0.99)) & (numerical.expected_annual_expenses > 0)
px.histogram(numerical[flag].expected_annual_expenses, height=500, width=750, labels={'value': "Annual Expenses in Retirement (Expected)", 'count': ""}).update_layout(showlegend=False)

In [55]:
flag = (numerical.debt_total <= numerical.debt_total.quantile(0.99)) & (numerical.debt_total >= 0)
px.histogram(numerical[flag].debt_total, height=500, width=750, labels={'value': "Total Debt", 'count': ""}).update_layout(showlegend=False)